In [144]:
import folium
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

%matplotlib inline

from gurobipy import *

In [2]:
path = "instances\InstanceBordeauxV1.xlsx"
# Employees
df_Workers = pd.read_excel(path, sheet_name=0, index_col='EmployeeName')
# Employees unvabilites
df_Workers_un = pd.read_excel(path, sheet_name=1, index_col='EmployeeName')
# Task
df_Task = pd.read_excel(path, sheet_name=2, index_col='TaskId')
# Task unvabilites
df_Task_un = pd.read_excel(path, sheet_name=3, index_col='TaskId')

In [3]:
# Employees
dict_Workers = df_Workers.to_dict('index')
# Employees unvabilites
dict_Workers_un = df_Workers_un.to_dict('index')
# Task
dict_Task = df_Task.to_dict('index')
# Task unvabilites
dict_Task_un = df_Task_un.to_dict('index')

In [207]:
dict_Workers

{'Valentin': {'Latitude': 45.15121765523164,
  'Longitude': -0.822092647754919,
  'Skill': 'Oenology',
  'Level': 2,
  'WorkingStartTime': '7:00am',
  'WorkingEndTime': '5:00pm'},
 'Ambre': {'Latitude': 45.19957452440505,
  'Longitude': -0.822092647754919,
  'Skill': 'Oenology',
  'Level': 1,
  'WorkingStartTime': '8:00am',
  'WorkingEndTime': '6:00pm'}}

In [208]:
dict_Task

{'T1': {'Latitude': 44.55654938342008,
  'Longitude': -0.3193922422375719,
  'TaskDuration': 60,
  'Skill': 'Oenology',
  'Level': 1,
  'OpeningTime': '8:00am',
  'ClosingTime': '6:00pm'},
 'T2': {'Latitude': 44.96750095217799,
  'Longitude': -0.6086852638150881,
  'TaskDuration': 60,
  'Skill': 'Oenology',
  'Level': 1,
  'OpeningTime': '8:00am',
  'ClosingTime': '6:00pm'},
 'T3': {'Latitude': 45.14421541464031,
  'Longitude': -0.7342570469020379,
  'TaskDuration': 60,
  'Skill': 'Oenology',
  'Level': 2,
  'OpeningTime': '8:00am',
  'ClosingTime': '6:00pm'},
 'T4': {'Latitude': 45.264808304867096,
  'Longitude': -0.7717887212411139,
  'TaskDuration': 60,
  'Skill': 'Oenology',
  'Level': 1,
  'OpeningTime': '12:00am',
  'ClosingTime': '6:00pm'},
 'T5': {'Latitude': 45.044422793402624,
  'Longitude': -0.6687606009488057,
  'TaskDuration': 60,
  'Skill': 'Oenology',
  'Level': 2,
  'OpeningTime': '8:00am',
  'ClosingTime': '6:00pm'},
 'T6': {'Latitude': 45.19957452440505,
  'Longitude'

#### Sets

In [4]:
Workers = list(df_Workers.index)
Skills = list(df_Workers["Skill"].unique())
Tasks = list(df_Task.index)
Houses = {w: "HouseOf" + w for w in Workers}

#### Parameters

level_skill: level of worker (in Workers) in some skill (of Skills)

In [36]:
def time_to_minutes(time_str):
    str_hour = time_str[:-2]
    am_pm = time_str[-2:]
    hour_str, minute = str_hour.split(':')
    hour = int(hour_str)
    if am_pm == 'pm':
        hour += 12
    return hour * 60 + int(minute)

def minutes_to_time(total_min):
    hour = int(total_min//60)
    min_ = int(total_min%60)
    am_pm = 'am'
    if hour > 12:
        am_pm = 'pm'
        hour -= 12
    if hour == 12:
        am_pm = 'pm'
    time = hour
    return f"{hour:02d}:{min_:02d}{am_pm}"

In [8]:
df_aux = pd.DataFrame()
for skill in Skills:
    df_aux[skill] = df_Workers.apply(lambda x: x['Level'] if x['Skill'] == skill else 0, axis=1)
l = df_aux.to_dict('index')
l

{'Valentin': {'Oenology': 2}, 'Ambre': {'Oenology': 1}}

In [9]:
# Opening time for taks i
a = df_Task.apply(lambda x: int(time_to_minutes(x['OpeningTime'])), axis=1).to_dict()
# Closing time for taks i
b = df_Task.apply(lambda x: int(time_to_minutes(x['ClosingTime'])), axis=1).to_dict()

# time worker w start working
alpha = df_Workers.apply(lambda x: int(time_to_minutes(x['WorkingStartTime'])), axis=1).to_dict()
# time worker w end working
beta = df_Workers.apply(lambda x: int(time_to_minutes(x['WorkingEndTime'])), axis=1).to_dict()

# Duration of the task i
d = df_Task['TaskDuration'].to_dict()
# Skill requierd by task i
s = df_Task['Skill'].to_dict()

In [10]:
# Level requierd by task i on the skill s
df_aux = pd.DataFrame()
for skill in Skills:
    df_aux[(skill)] = df_Task.apply(lambda x: x['Level'] if x['Skill'] == skill else 100, axis=1)
r = df_aux.to_dict('index')
r

{'T1': {'Oenology': 1},
 'T2': {'Oenology': 1},
 'T3': {'Oenology': 2},
 'T4': {'Oenology': 1},
 'T5': {'Oenology': 2},
 'T6': {'Oenology': 2},
 'T7': {'Oenology': 1},
 'T8': {'Oenology': 1},
 'T9': {'Oenology': 2},
 'T10': {'Oenology': 1}}

In [11]:
def read_lat_log(df, alias = None):
    nodes = {}
    for name, dic_inf in df.items():
        if alias:
            name = alias[name]
        nodes[name] = (dic_inf["Latitude"], dic_inf["Longitude"])
    return nodes

def haversine(pt1, pt2):
    R = 6371  # radius of the Earth in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [pt1[0], pt1[1], pt2[0], pt2[1]])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

def distance_matrix(nodes):
    # Create distance matrix with haversine distance
    dist_matrix = {}
    for node_i in nodes.keys():
        dist_matrix[node_i] = {}
        for node_j in nodes.keys():
            dist = haversine(nodes[node_i], nodes[node_j])
            dist_matrix[node_i][node_j] = int(np.ceil((dist / 50) * 60))
    return dist_matrix


In [12]:
# Denfine the nodes
nodes = read_lat_log(dict_Workers, Houses) | read_lat_log(dict_Task)

## Define the time matrix in minutes round ceil between a node (task or worker) with another (task or worker)
t = distance_matrix(nodes)
t['HouseOfValentin']['T1']

93

In [13]:
def init_param():
    print(Workers)
    print(Skills)
    print(Tasks)
    print(Houses)
    print(l)
    print(a)
    print(b)
    print(alpha)
    print(beta)
    print(d)
    print(s)
    print(r)
    print(t)

init_param()

['Valentin', 'Ambre']
['Oenology']
['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10']
{'Valentin': 'HouseOfValentin', 'Ambre': 'HouseOfAmbre'}
{'Valentin': {'Oenology': 2}, 'Ambre': {'Oenology': 1}}
{'T1': 480, 'T2': 480, 'T3': 480, 'T4': 720, 'T5': 480, 'T6': 480, 'T7': 480, 'T8': 480, 'T9': 480, 'T10': 480}
{'T1': 1080, 'T2': 1080, 'T3': 1080, 'T4': 1080, 'T5': 1080, 'T6': 1080, 'T7': 1080, 'T8': 1080, 'T9': 1440, 'T10': 1080}
{'Valentin': 420, 'Ambre': 480}
{'Valentin': 1020, 'Ambre': 1080}
{'T1': 60, 'T2': 60, 'T3': 60, 'T4': 60, 'T5': 60, 'T6': 60, 'T7': 60, 'T8': 60, 'T9': 75, 'T10': 60}
{'T1': 'Oenology', 'T2': 'Oenology', 'T3': 'Oenology', 'T4': 'Oenology', 'T5': 'Oenology', 'T6': 'Oenology', 'T7': 'Oenology', 'T8': 'Oenology', 'T9': 'Oenology', 'T10': 'Oenology'}
{'T1': {'Oenology': 1}, 'T2': {'Oenology': 1}, 'T3': {'Oenology': 2}, 'T4': {'Oenology': 1}, 'T5': {'Oenology': 2}, 'T6': {'Oenology': 2}, 'T7': {'Oenology': 1}, 'T8': {'Oenology': 1}, 'T9': {'Oenology': 2}

### Les variables

In [147]:
def init_param():
    return Workers, Skills, Tasks, l, a, b, alpha, beta, d, s, r, t

## initialisation du modèle
m = Model("Phase 1")


## initialisation des variables 
X = {(i, j, w): m.addVar(vtype=GRB.BINARY, name=f"{w}_fait_le_trajet_{i}_à_{j}") for w in Workers for j in Tasks for i in Tasks if j != i} |\
      {(i, Houses[w], w): m.addVar(vtype=GRB.BINARY, name=f"{w}_fait_le_trajet_{i}_à_{Houses[w]}") for w in Workers for i in Tasks} |\
          {(Houses[w], j, w): m.addVar(vtype=GRB.BINARY, name=f"{w}_fait_le_trajet_{Houses[w]}_à_{j}") for j in Tasks for w in Workers} |\
              {(Houses[w], Houses[w], w): m.addVar(vtype=GRB.BINARY, name=f"{w}_fait_le_trajet_{Houses[w]}_à_{Houses[w]}") for w in Workers}

T = {i: m.addVar(vtype=GRB.INTEGER, name=f"temps_début_tâche_{i}") for i in Tasks}

## Variables additionnelles

Y = {(i, w): LinExpr(quicksum([X[(i, j, w)] for j in Tasks + [Houses[w]] if j != i])) for i in Tasks for w in Workers}
Y_bis = {(i, w): LinExpr(quicksum([X[(j, i, w)] for j in Tasks + [Houses[w]] if j != i])) for i in Tasks for w in Workers}

## initialisation des contraintes

## 1- All tasks have to be done once
ContrDone = {i: m.addConstr(quicksum([Y[(i, w)] for w in Workers])  == 1) for i in Tasks}

## 2 -Workers have to be capable of doing the Tasks
MS = 10
ContrSkill = {(i, w, s):m.addConstr(r[i][s] <= l[w][s] + MS*(1 - Y[(i, w)])) for i in Tasks for w in Workers for s in Skills}

## 3- flow restriction
ContrFlow = {(i, w): m.addConstr(Y[(i, w)] == Y_bis[(i, w)]) for i in Tasks for w in Workers}

## 4- Border flow conditions
ContrBorderL = {w: m.addConstr(quicksum([X[(i, Houses[w], w)] for i in Tasks + [Houses[w]]]) == 1) for w in Workers}
ContrBorderR = {w: m.addConstr(quicksum([X[(Houses[w], j, w)] for j in Tasks + [Houses[w]]]) == 1) for w in Workers}

## 5- Task disponibility
ContrTaskDisp = {i: m.addConstr(a[i] <= T[i]) for i in Tasks}
ContrTaskDisp = {i: m.addConstr(T[i] + d[i] <= b[i]) for i in Tasks}

## 6- task sequence is possible
MT = 24*60
ContrSeq = {(i, j, w): m.addConstr(T[i] + d[i] + t[i][j] <= T[j] + MT*(1 - X[(i, j, w)])) for i in Tasks for j in Tasks for w in Workers if i != j}

## 7- Border task sequence conditions
ContrBorderSeqDeb = {(Houses[w], j, w): m.addConstr(alpha[w] + t[Houses[w]][j] <= T[j] + MT*(1 - X[(Houses[w], j, w)])) for j in Tasks for w in Workers}
ContrBorderSeqFin = {(i, Houses[w], w): m.addConstr(T[i] + d[i] + t[i][Houses[w]] <= beta[w] + MT*(1 - X[(i, Houses[w], w)])) for i in Tasks for w in Workers}

## 8- Employees have unavailabilities


## Initialisation de l'objectif
m.setObjective(quicksum([t[i][j]*X[(i, j, w)] for (i, j, w) in X.keys()]), GRB.MINIMIZE)

m.update()
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 294 rows, 232 columns and 1484 nonzeros
Model fingerprint: 0x98bbebe1
Variable types: 0 continuous, 232 integer (222 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 128 rows and 68 columns
Presolve time: 0.02s
Presolved: 166 rows, 164 columns, 802 nonzeros
Variable types: 0 continuous, 164 integer (154 binary)

Root relaxation: objective 2.226646e+02, 42 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  222.66460    0   20          -  222.66460 

In [148]:
for (i, j, w) in X.keys():
    if X[(i, j, w)].x:
        if j in Tasks:
            print(X[(i, j, w)].VarName, "à", minutes_to_time(T[j].x))
        else:
             print(X[(i, j, w)].VarName)

Valentin_fait_le_trajet_T6_à_T3 à 02:18pm
Valentin_fait_le_trajet_T7_à_T4 à 12:00pm
Valentin_fait_le_trajet_T3_à_T5 à 03:39pm
Valentin_fait_le_trajet_T4_à_T6 à 01:10pm
Valentin_fait_le_trajet_T9_à_T7 à 10:34am
Ambre_fait_le_trajet_T2_à_T1 à 10:39am
Ambre_fait_le_trajet_T1_à_T8 à 03:36pm
Ambre_fait_le_trajet_T8_à_T10 à 04:44pm
Valentin_fait_le_trajet_T5_à_HouseOfValentin
Ambre_fait_le_trajet_T10_à_HouseOfAmbre
Ambre_fait_le_trajet_HouseOfAmbre_à_T2 à 08:37am
Valentin_fait_le_trajet_HouseOfValentin_à_T9 à 09:04am


In [149]:
print(['taskId', 'performed', 'employeeName', 'startTime'])

# all_rutes = {worker:['HouseOf'+str(worker)] for worker in Workers}
all_rutes = {worker:[] for worker in Workers}
for taks in T.keys():
    result = [taks]
    if T[taks].x == 0:
        result.append(0)
        result += ['','']
    else:
        result.append(1)
        for (i, j, w) in X.keys():
            if i == taks:
                if X[(i, j, w)].x:
                    result.append(w)
                    all_rutes[w].append(result)
        result.append(int(T[taks].x))
    print(result)

['taskId', 'performed', 'employeeName', 'startTime']
['T1', 1, 'Ambre', 639]
['T2', 1, 'Ambre', 517]
['T3', 1, 'Valentin', 858]
['T4', 1, 'Valentin', 720]
['T5', 1, 'Valentin', 939]
['T6', 1, 'Valentin', 790]
['T7', 1, 'Valentin', 634]
['T8', 1, 'Ambre', 936]
['T9', 1, 'Valentin', 544]
['T10', 1, 'Ambre', 1004]


In [150]:
# sort the routes in order of time for each worker
for w in all_rutes.keys():
    all_rutes[w] = sorted(all_rutes[w], key=lambda x: x[3])
    
# get the coordinates of the route for each worker
routes_lat_log = {worker:[] for worker in Workers}
for w in Workers:
    routes_lat_log[w].append(nodes['HouseOf'+w])
    routes_lat_log[w] += [nodes[task[0]] for task in all_rutes[w]]
    routes_lat_log[w].append(nodes['HouseOf'+w])


In [152]:
# create a map centered on the first point of the first route
m = folium.Map(location=nodes['HouseOf'+Workers[0]], zoom_start=10)

# add markers for each point in the first route
for node in nodes.values():
    folium.Marker(location=node).add_to(m)

# add a polyline to connect the markers for the second route
for route in routes_lat_log.values():
    color = '#{:06x}'.format(random.randint(0, 0xFFFFFF))
    folium.PolyLine(locations=route, color=color, weight=4).add_to(m)

# show the map
m
